In [16]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import os
import re
import ast

data_path = '/Users/amanda/Documents/Projects/insight/data/'

In [17]:
def convert_to_list(df,columns):

    df.fillna('', inplace=True)

    for col in columns:
        if isinstance(df[col][0], str):
            df[col] = [ast.literal_eval(s) for s in df[col]]

    return df

In [18]:
headers = {'User-Agent': 'Mozilla/5.0',
           'From': 'itchyandscratchy@gmail.com'
          }

In [19]:
# Load CIP codes and levels of education

code_df = pd.read_csv(os.path.join(data_path,'raw','cip-codes.csv'))
cip_codes = code_df['CIP Code'].str.strip("=").str.strip('"')
cip_list = cip_codes.tolist()

levels = ['LOS03','LOS04','LOS05','LOS06','LOS07']

In [ ]:
# Try urls associated with each CIP code to find out if they exist
# Note this is a long process and only generally needs to be run once

url_list = []

for i in cip_list[0:10]:
    for j in levels:
    
        url = 'https://www.jobbank.gc.ca/studentdashboard/{}/{}'.format(i,j)
        print(url)
        request = requests.get(url, headers=headers, allow_redirects=False)
        if request.status_code == 200:
            url_list.append(url)

In [10]:
# Save url_list to file so I can load it later

file_path = (os.path.join(data_path,'processed','url_list.txt'))

with open(file_path, 'w') as filehandle:
    for listitem in url_list:
        filehandle.write('%s\n' % listitem)

https://www.jobbank.gc.ca/studentdashboard/15.1307/LOS03
https://www.jobbank.gc.ca/studentdashboard/15.1307/LOS04
https://www.jobbank.gc.ca/studentdashboard/15.1307/LOS05
https://www.jobbank.gc.ca/studentdashboard/15.1307/LOS06
https://www.jobbank.gc.ca/studentdashboard/15.1307/LOS07
https://www.jobbank.gc.ca/studentdashboard/61.2603/LOS03
https://www.jobbank.gc.ca/studentdashboard/61.2603/LOS04
https://www.jobbank.gc.ca/studentdashboard/61.2603/LOS05
https://www.jobbank.gc.ca/studentdashboard/61.2603/LOS06
https://www.jobbank.gc.ca/studentdashboard/61.2603/LOS07
https://www.jobbank.gc.ca/studentdashboard/32.0112/LOS03
https://www.jobbank.gc.ca/studentdashboard/32.0112/LOS04
https://www.jobbank.gc.ca/studentdashboard/32.0112/LOS05
https://www.jobbank.gc.ca/studentdashboard/32.0112/LOS06
https://www.jobbank.gc.ca/studentdashboard/32.0112/LOS07
https://www.jobbank.gc.ca/studentdashboard/52.0305/LOS03
https://www.jobbank.gc.ca/studentdashboard/52.0305/LOS04
https://www.jobbank.gc.ca/stude

https://www.jobbank.gc.ca/studentdashboard/53.0105/LOS04
https://www.jobbank.gc.ca/studentdashboard/53.0105/LOS05
https://www.jobbank.gc.ca/studentdashboard/53.0105/LOS06
https://www.jobbank.gc.ca/studentdashboard/53.0105/LOS07
https://www.jobbank.gc.ca/studentdashboard/13.1502/LOS03
https://www.jobbank.gc.ca/studentdashboard/13.1502/LOS04
https://www.jobbank.gc.ca/studentdashboard/13.1502/LOS05
https://www.jobbank.gc.ca/studentdashboard/13.1502/LOS06
https://www.jobbank.gc.ca/studentdashboard/13.1502/LOS07
https://www.jobbank.gc.ca/studentdashboard/60.0704/LOS03
https://www.jobbank.gc.ca/studentdashboard/60.0704/LOS04
https://www.jobbank.gc.ca/studentdashboard/60.0704/LOS05
https://www.jobbank.gc.ca/studentdashboard/60.0704/LOS06
https://www.jobbank.gc.ca/studentdashboard/60.0704/LOS07
https://www.jobbank.gc.ca/studentdashboard/60.0705/LOS03
https://www.jobbank.gc.ca/studentdashboard/60.0705/LOS04
https://www.jobbank.gc.ca/studentdashboard/60.0705/LOS05
https://www.jobbank.gc.ca/stude

https://www.jobbank.gc.ca/studentdashboard/01.0401/LOS04
https://www.jobbank.gc.ca/studentdashboard/01.0401/LOS05
https://www.jobbank.gc.ca/studentdashboard/01.0401/LOS06
https://www.jobbank.gc.ca/studentdashboard/01.0401/LOS07
https://www.jobbank.gc.ca/studentdashboard/01.1104/LOS03
https://www.jobbank.gc.ca/studentdashboard/01.1104/LOS04
https://www.jobbank.gc.ca/studentdashboard/01.1104/LOS05
https://www.jobbank.gc.ca/studentdashboard/01.1104/LOS06
https://www.jobbank.gc.ca/studentdashboard/01.1104/LOS07
https://www.jobbank.gc.ca/studentdashboard/01.0902/LOS03
https://www.jobbank.gc.ca/studentdashboard/01.0902/LOS04
https://www.jobbank.gc.ca/studentdashboard/01.0902/LOS05
https://www.jobbank.gc.ca/studentdashboard/01.0902/LOS06
https://www.jobbank.gc.ca/studentdashboard/01.0902/LOS07
https://www.jobbank.gc.ca/studentdashboard/01.0101/LOS03
https://www.jobbank.gc.ca/studentdashboard/01.0101/LOS04
https://www.jobbank.gc.ca/studentdashboard/01.0101/LOS05
https://www.jobbank.gc.ca/stude

https://www.jobbank.gc.ca/studentdashboard/01.1399/LOS06
https://www.jobbank.gc.ca/studentdashboard/01.1399/LOS07
https://www.jobbank.gc.ca/studentdashboard/01.13/LOS03
https://www.jobbank.gc.ca/studentdashboard/01.13/LOS04
https://www.jobbank.gc.ca/studentdashboard/01.13/LOS05
https://www.jobbank.gc.ca/studentdashboard/01.13/LOS06
https://www.jobbank.gc.ca/studentdashboard/01.13/LOS07
https://www.jobbank.gc.ca/studentdashboard/01.0308/LOS03
https://www.jobbank.gc.ca/studentdashboard/01.0308/LOS04
https://www.jobbank.gc.ca/studentdashboard/01.0308/LOS05
https://www.jobbank.gc.ca/studentdashboard/01.0308/LOS06
https://www.jobbank.gc.ca/studentdashboard/01.0308/LOS07
https://www.jobbank.gc.ca/studentdashboard/01.1102/LOS03
https://www.jobbank.gc.ca/studentdashboard/01.1102/LOS04
https://www.jobbank.gc.ca/studentdashboard/01.1102/LOS05
https://www.jobbank.gc.ca/studentdashboard/01.1102/LOS06
https://www.jobbank.gc.ca/studentdashboard/01.1102/LOS07
https://www.jobbank.gc.ca/studentdashboar

https://www.jobbank.gc.ca/studentdashboard/51.09/LOS07
https://www.jobbank.gc.ca/studentdashboard/51.3499/LOS03
https://www.jobbank.gc.ca/studentdashboard/51.3499/LOS04
https://www.jobbank.gc.ca/studentdashboard/51.3499/LOS05
https://www.jobbank.gc.ca/studentdashboard/51.3499/LOS06
https://www.jobbank.gc.ca/studentdashboard/51.3499/LOS07
https://www.jobbank.gc.ca/studentdashboard/51.34/LOS03
https://www.jobbank.gc.ca/studentdashboard/51.34/LOS04
https://www.jobbank.gc.ca/studentdashboard/51.34/LOS05
https://www.jobbank.gc.ca/studentdashboard/51.34/LOS06
https://www.jobbank.gc.ca/studentdashboard/51.34/LOS07
https://www.jobbank.gc.ca/studentdashboard/51.3300/LOS03
https://www.jobbank.gc.ca/studentdashboard/51.3300/LOS04
https://www.jobbank.gc.ca/studentdashboard/51.3300/LOS05
https://www.jobbank.gc.ca/studentdashboard/51.3300/LOS06
https://www.jobbank.gc.ca/studentdashboard/51.3300/LOS07
https://www.jobbank.gc.ca/studentdashboard/51.3399/LOS03
https://www.jobbank.gc.ca/studentdashboard/

...

https://www.jobbank.gc.ca/studentdashboard/13.0410/LOS05
https://www.jobbank.gc.ca/studentdashboard/13.0410/LOS06
https://www.jobbank.gc.ca/studentdashboard/13.0410/LOS07
https://www.jobbank.gc.ca/studentdashboard/03.0508/LOS03
https://www.jobbank.gc.ca/studentdashboard/03.0508/LOS04
https://www.jobbank.gc.ca/studentdashboard/03.0508/LOS05
https://www.jobbank.gc.ca/studentdashboard/03.0508/LOS06
https://www.jobbank.gc.ca/studentdashboard/03.0508/LOS07
https://www.jobbank.gc.ca/studentdashboard/39.0703/LOS03
https://www.jobbank.gc.ca/studentdashboard/39.0703/LOS04
https://www.jobbank.gc.ca/studentdashboard/39.0703/LOS05
https://www.jobbank.gc.ca/studentdashboard/39.0703/LOS06
https://www.jobbank.gc.ca/studentdashboard/39.0703/LOS07
https://www.jobbank.gc.ca/studentdashboard/45.12/LOS03
https://www.jobbank.gc.ca/studentdashboard/45.12/LOS04
https://www.jobbank.gc.ca/studentdashboard/45.12/LOS05
https://www.jobbank.gc.ca/studentdashboard/45.12/LOS06
https://www.jobbank.gc.ca/studentdashbo

https://www.jobbank.gc.ca/studentdashboard/01.8105/LOS05
https://www.jobbank.gc.ca/studentdashboard/01.8105/LOS06
https://www.jobbank.gc.ca/studentdashboard/01.8105/LOS07
https://www.jobbank.gc.ca/studentdashboard/60.0301/LOS03
https://www.jobbank.gc.ca/studentdashboard/60.0301/LOS04
https://www.jobbank.gc.ca/studentdashboard/60.0301/LOS05
https://www.jobbank.gc.ca/studentdashboard/60.0301/LOS06
https://www.jobbank.gc.ca/studentdashboard/60.0301/LOS07
https://www.jobbank.gc.ca/studentdashboard/60.0319/LOS03
https://www.jobbank.gc.ca/studentdashboard/60.0319/LOS04
https://www.jobbank.gc.ca/studentdashboard/60.0319/LOS05
https://www.jobbank.gc.ca/studentdashboard/60.0319/LOS06
https://www.jobbank.gc.ca/studentdashboard/60.0319/LOS07
https://www.jobbank.gc.ca/studentdashboard/01.8199/LOS03
https://www.jobbank.gc.ca/studentdashboard/01.8199/LOS04
https://www.jobbank.gc.ca/studentdashboard/01.8199/LOS05
https://www.jobbank.gc.ca/studentdashboard/01.8199/LOS06
https://www.jobbank.gc.ca/stude

https://www.jobbank.gc.ca/studentdashboard/60.0311/LOS05
https://www.jobbank.gc.ca/studentdashboard/60.0311/LOS06
https://www.jobbank.gc.ca/studentdashboard/60.0311/LOS07
https://www.jobbank.gc.ca/studentdashboard/60.0312/LOS03
https://www.jobbank.gc.ca/studentdashboard/60.0312/LOS04
https://www.jobbank.gc.ca/studentdashboard/60.0312/LOS05
https://www.jobbank.gc.ca/studentdashboard/60.0312/LOS06
https://www.jobbank.gc.ca/studentdashboard/60.0312/LOS07
https://www.jobbank.gc.ca/studentdashboard/01.8110/LOS03
https://www.jobbank.gc.ca/studentdashboard/01.8110/LOS04
https://www.jobbank.gc.ca/studentdashboard/01.8110/LOS05
https://www.jobbank.gc.ca/studentdashboard/01.8110/LOS06
https://www.jobbank.gc.ca/studentdashboard/01.8110/LOS07
https://www.jobbank.gc.ca/studentdashboard/51.2510/LOS03
https://www.jobbank.gc.ca/studentdashboard/51.2510/LOS04
https://www.jobbank.gc.ca/studentdashboard/51.2510/LOS05
https://www.jobbank.gc.ca/studentdashboard/51.2510/LOS06
https://www.jobbank.gc.ca/stude

https://www.jobbank.gc.ca/studentdashboard/51.2310/LOS05
https://www.jobbank.gc.ca/studentdashboard/51.2310/LOS06
https://www.jobbank.gc.ca/studentdashboard/51.2310/LOS07
https://www.jobbank.gc.ca/studentdashboard/50.0908/LOS03
https://www.jobbank.gc.ca/studentdashboard/50.0908/LOS04
https://www.jobbank.gc.ca/studentdashboard/50.0908/LOS05
https://www.jobbank.gc.ca/studentdashboard/50.0908/LOS06
https://www.jobbank.gc.ca/studentdashboard/50.0908/LOS07
https://www.jobbank.gc.ca/studentdashboard/10.0204/LOS03
https://www.jobbank.gc.ca/studentdashboard/10.0204/LOS04
https://www.jobbank.gc.ca/studentdashboard/10.0204/LOS05
https://www.jobbank.gc.ca/studentdashboard/10.0204/LOS06
https://www.jobbank.gc.ca/studentdashboard/10.0204/LOS07
https://www.jobbank.gc.ca/studentdashboard/13.1208/LOS03
https://www.jobbank.gc.ca/studentdashboard/13.1208/LOS04
https://www.jobbank.gc.ca/studentdashboard/13.1208/LOS05
https://www.jobbank.gc.ca/studentdashboard/13.1208/LOS06
https://www.jobbank.gc.ca/stude

https://www.jobbank.gc.ca/studentdashboard/48.0704/LOS05
https://www.jobbank.gc.ca/studentdashboard/48.0704/LOS06
https://www.jobbank.gc.ca/studentdashboard/48.0704/LOS07
https://www.jobbank.gc.ca/studentdashboard/50.0915/LOS03
https://www.jobbank.gc.ca/studentdashboard/50.0915/LOS04
https://www.jobbank.gc.ca/studentdashboard/50.0915/LOS05
https://www.jobbank.gc.ca/studentdashboard/50.0915/LOS06
https://www.jobbank.gc.ca/studentdashboard/50.0915/LOS07
https://www.jobbank.gc.ca/studentdashboard/48.0701/LOS03
https://www.jobbank.gc.ca/studentdashboard/48.0701/LOS04
https://www.jobbank.gc.ca/studentdashboard/48.0701/LOS05
https://www.jobbank.gc.ca/studentdashboard/48.0701/LOS06
https://www.jobbank.gc.ca/studentdashboard/48.0701/LOS07
https://www.jobbank.gc.ca/studentdashboard/48.0799/LOS03
https://www.jobbank.gc.ca/studentdashboard/48.0799/LOS04
https://www.jobbank.gc.ca/studentdashboard/48.0799/LOS05
https://www.jobbank.gc.ca/studentdashboard/48.0799/LOS06
https://www.jobbank.gc.ca/stude

In [20]:
# Load url_list if not in memory

file_path = (os.path.join(data_path,'processed','url_list.txt'))

# define empty list
url_list = []

# open file and read the content in a list
with open(file_path, 'r') as filehandle:
    url_list = [current_place.rstrip() for current_place in filehandle.readlines()]

# Scrape education and associated jobs data

In [13]:
# Describe dataframe up front and add a row on each loop iteration to ensure 
# rows are always complete and there's no missing data resulting in problems
# at the end

jobs_df = pd.DataFrame({'edu_level':[],
                        'degree':[],
                        'description':[],
                        'employed':[],
                        'unemployed':[],
                        'med_earnings':[],
                        'range_earnings':[],
                        'related_field':[],
                        'unrelated_field':[],
                        'relation':[],
                        'top_jobs':[],
                        'top_job_links':[],
                        'job_percent':[],
                        'field_satisfaction':[]})

for url in url_list:
    
    try:
    
        with requests.get(url, timeout=5, headers=headers) as u:     
            soup = BeautifulSoup(u.content) 

            # the following is very specific to the website
            for a in soup.findAll('div', attrs={'class':'job-market'}):

                try:
                    # Add any interested attributes here
                    d_type = a.find('span', attrs={'class':'section-name'})
                    sec = a.find('span', attrs={'class':'heading-info'})
                    description = a.find('p', attrs={'class':'intro margin-top1 margin-bottom2'})

                    print(sec.text.strip()+':'+d_type.text.strip())

                    # Individual sections of the page described below
                    employment = a.find('div', attrs={'id':'employment_outcomes'})
                    p_employed = employment.find('p', attrs={'class':'stats_number_big'})
                    p_unemployed = employment.findAll('p', attrs={'class':'stats_number'})

                    earns = a.find('div', attrs={'class':'earning-section'})
                    med_earn = earns.find('p', attrs={'class':'stats_number_big'})
                    range_earn = earns.find('p', attrs={'class':'stats_number'})

                    related = a.find('div', attrs={'class':'related-section'})
                    close = related.find('p', attrs={'class':'stats_number_big'})
                    some_not = related.findAll('p', attrs={'class':'stats_number'}) 
                    how_related = [s.find_next_sibling('p') for s in some_not]

                    occupation = a.find('div', attrs={'class':'occupation-section'})
                    job_name = occupation.findAll('span', attrs={'class':'text-center'})
                    job_links = occupation.findAll('a', href=True)
                    job_percent = occupation.findAll('td', attrs={'headers':'top_ten_percent'})

                    satis = a.find('div', attrs={'class':'satisfaction-section'})
                    stud_again = satis.findAll('p', attrs={'class':'stats_number'})  

                except:               
                    continue

                # Organize attributes to append to dataframe
                if d_type is not None and sec is not None and description is not None:
                    degree_types = d_type.text.strip()
                    categories = sec.text.strip()
                    facts = description.text.strip()
                else:
                    continue

                if p_employed is not None and p_unemployed is not None:
                    stats_employed = p_employed.text.strip()
                    stats_unemployed = [s.text.strip() for s in p_unemployed]
                else:
                    stats_employed = np.NaN
                    stats_unemployed = np.NaN

                if med_earn is not None and range_earn is not None:
                    med_earnings = med_earn.text.strip()
                    range_earnings = range_earn.text.strip()
                else:
                    med_earnings = np.NaN
                    range_earnings = np.NaN

                if close:
                    related_fields = close.text.strip()
                else:
                    related_fields = np.NaN

                if some_not:
                    unrelated_fields = [s.text.strip() for s in some_not]
                    relation = [s.text.strip() for s in how_related]
                else:
                    unrelated_fields = np.NaN
                    relation = np.NaN

                if job_name is not None and job_links is not None and job_percent is not None:
                    if len(job_name) == len(job_links) == len(job_percent):
                        top_jobs = [s.text.strip() for s in job_name]
                        link_top_jobs = [s.get('href') for s in job_links]
                        per_top_jobs = [s.text.strip() for s in job_percent]
                else:
                    top_jobs = np.NaN
                    link_top_jobs = np.NaN
                    per_top_jobs = np.NaN

                if stud_again:
                    satisfaction = [s.text.strip() for s in stud_again]
                else:
                    satisfaction = np.NaN

                try:                
                    jobs_df = jobs_df.append({'edu_level':degree_types,
                                              'degree':categories,
                                              'description':facts,
                                              'employed':stats_employed,
                                              'unemployed':stats_unemployed,
                                              'med_earnings':med_earnings,
                                              'range_earnings':range_earnings,
                                              'related_field':related_fields,
                                              'unrelated_field':unrelated_fields,
                                              'relation':relation,
                                              'top_jobs':top_jobs,
                                              'top_job_links':link_top_jobs,
                                              'job_percent':per_top_jobs,
                                              'field_satisfaction':satisfaction}, ignore_index=True)

                except:
                    print('NOT APPENDED')
                    continue
    except:
        continue

Accounting and Business/Management:College/CEGEP
Accounting and Business/Management:Bachelor's degree
Accounting and Business/Management:Master's degree
Accounting and Computer Science:Other trades certificate or diploma
Accounting and Computer Science:College/CEGEP
Accounting and Computer Science:Bachelor's degree
Accounting and Computer Science:Master's degree
Accounting and Finance:College/CEGEP
Accounting and Finance:Bachelor's degree
Accounting and Finance:Master's degree
Accounting and Related Services, Other:College/CEGEP
Accounting and Related Services, Other:Bachelor's degree
Accounting and Related Services, Other:Master's degree
Accounting Technology/Technician and Bookkeeping:College/CEGEP
Accounting Technology/Technician and Bookkeeping:Bachelor's degree
Accounting Technology/Technician and Bookkeeping:Master's degree
Accounting:College/CEGEP
Accounting:Bachelor's degree
Accounting:Master's degree
Acoustics:College/CEGEP
Acoustics:Bachelor's degree
Acoustics:Master's degree

Agricultural Production Operations, General:Master's degree
Agricultural Production Operations, Other:Other trades certificate or diploma
Agricultural Production Operations, Other:College/CEGEP
Agricultural Production Operations, Other:Bachelor's degree
Agricultural Production Operations, Other:Master's degree
Agricultural Public Services, Other:Other trades certificate or diploma
Agricultural Public Services, Other:College/CEGEP
Agricultural Public Services, Other:Bachelor's degree
Agricultural Public Services, Other:Master's degree
Agricultural Teacher Education:College/CEGEP
Agricultural Teacher Education:Bachelor's degree
Agricultural Teacher Education:Master's degree
Agriculture, Agriculture Operations and Related Sciences, Other:Other trades certificate or diploma
Agriculture, Agriculture Operations and Related Sciences, Other:College/CEGEP
Agriculture, Agriculture Operations and Related Sciences, Other:Bachelor's degree
Agriculture, Agriculture Operations and Related Sciences, O

Apparel and Textiles, General:Bachelor's degree
Apparel and Textiles, General:Master's degree
Family and Consumer Sciences/Human Sciences, Other:Bachelor's degree
Family and Consumer Sciences/Human Sciences, Other:Master's degree
Appliance Installation and Repair Technology/Technician:Other trades certificate or diploma
Appliance Installation and Repair Technology/Technician:College/CEGEP
Applied Economics:College/CEGEP
Applied Economics:Bachelor's degree
Applied Economics:Master's degree
Applied Horticulture/Horticultural Business Services, Other:Other trades certificate or diploma
Applied Horticulture/Horticultural Business Services, Other:College/CEGEP
Applied Horticulture/Horticultural Business Services, Other:Bachelor's degree
Applied Horticulture/Horticultural Business Services, Other:Master's degree
Applied Horticulture/Horticulture Operations, General:Other trades certificate or diploma
Applied Horticulture/Horticulture Operations, General:College/CEGEP
Applied Horticulture/Hor

Baltic Studies:Master's degree
Banking and Financial Support Services:College/CEGEP
Banking and Financial Support Services:Bachelor's degree
Banking and Financial Support Services:Master's degree
Banking, Corporate, Finance and Securities Law (LLM, JSD/SJD):College/CEGEP
Banking, Corporate, Finance and Securities Law (LLM, JSD/SJD):Bachelor's degree
Banking, Corporate, Finance and Securities Law (LLM, JSD/SJD):Master's degree
Barbering/Barber:Other trades certificate or diploma
Barbering/Barber:College/CEGEP
Bartending/Bartender:Other trades certificate or diploma
Bartending/Bartender:College/CEGEP
Behavioural Sciences:College/CEGEP
Behavioural Sciences:Bachelor's degree
Behavioural Sciences:Master's degree
Bengali Language and Literature:College/CEGEP
Bengali Language and Literature:Bachelor's degree
Bengali Language and Literature:Master's degree
Bible/Biblical Studies:College/CEGEP
Bible/Biblical Studies:Bachelor's degree
Bible/Biblical Studies:Master's degree
Bicycle Mechanics and 

CAD/CADD Drafting and/or Design Technology/Technician:College/CEGEP
Canadian Government and Politics:College/CEGEP
Canadian Government and Politics:Bachelor's degree
Canadian Government and Politics:Master's degree
Canadian History:College/CEGEP
Canadian History:Bachelor's degree
Canadian History:Master's degree
Canadian Law/Legal Studies/Jurisprudence (LLM, MCJ, JSD/SJD):College/CEGEP
Canadian Law/Legal Studies/Jurisprudence (LLM, MCJ, JSD/SJD):Bachelor's degree
Canadian Law/Legal Studies/Jurisprudence (LLM, MCJ, JSD/SJD):Master's degree
Canadian Studies:College/CEGEP
Canadian Studies:Bachelor's degree
Canadian Studies:Master's degree
Cardiopulmonary Technology/Technologist:College/CEGEP
Cardiopulmonary Technology/Technologist:Bachelor's degree
Cardiopulmonary Technology/Technologist:Master's degree
Cardiovascular Science:College/CEGEP
Cardiovascular Science:Bachelor's degree
Cardiovascular Science:Master's degree
Cardiovascular Technology/Technologist:College/CEGEP
Cardiovascular Tec

...

Reproductive Biology:Bachelor's degree
Reproductive Biology:Master's degree
French as a Second Language:College/CEGEP
French as a Second Language:Bachelor's degree
French as a Second Language:Master's degree
French Language and Literature, General:College/CEGEP
French Language and Literature, General:Bachelor's degree
French Language and Literature, General:Master's degree
Second Language Learning, Other:College/CEGEP
Second Language Learning, Other:Bachelor's degree
Second Language Learning, Other:Master's degree
Technology Education/Industrial Arts Programs:College/CEGEP
French Language and Literature/Letters, Other:College/CEGEP
French Language and Literature/Letters, Other:Bachelor's degree
French Language and Literature/Letters, Other:Master's degree
Resort Management:College/CEGEP
Resort Management:Bachelor's degree
Resort Management:Master's degree
Respiratory Care Therapy/Therapist:College/CEGEP
Respiratory Care Therapy/Therapist:Bachelor's degree
Respiratory Care Therapy/Thera

Social Work, Other:Other trades certificate or diploma
Social Work, Other:College/CEGEP
Social Work, Other:Bachelor's degree
Social Work, Other:Master's degree
Social Work, General:Other trades certificate or diploma
Social Work, General:College/CEGEP
Social Work, General:Bachelor's degree
Social Work, General:Master's degree
Sociology:College/CEGEP
Sociology:Bachelor's degree
Sociology:Master's degree
Soil Chemistry and Physics:Other trades certificate or diploma
Soil Chemistry and Physics:College/CEGEP
Soil Chemistry and Physics:Bachelor's degree
Soil Chemistry and Physics:Master's degree
Soil Microbiology:Other trades certificate or diploma
Soil Microbiology:College/CEGEP
Soil Microbiology:Bachelor's degree
Soil Microbiology:Master's degree
Soil Science and Agronomy, General:Other trades certificate or diploma
Soil Science and Agronomy, General:College/CEGEP
Soil Science and Agronomy, General:Bachelor's degree
Soil Science and Agronomy, General:Master's degree
Soil Sciences, Other:O

Teacher Education and Professional Development, Specific Levels and Methods, Other:Master's degree
Teacher Education and Professional Development, Specific Subject Areas, Other:College/CEGEP
Teacher Education and Professional Development, Specific Subject Areas, Other:Bachelor's degree
Teacher Education and Professional Development, Specific Subject Areas, Other:Master's degree
Teacher Education, Multiple Levels:College/CEGEP
Teacher Education, Multiple Levels:Bachelor's degree
Teacher Education, Multiple Levels:Master's degree
Teaching Assistants/Aides, Other:College/CEGEP
Teaching Assistants/Aides, Other:Bachelor's degree
Teaching Assistants/Aides, Other:Master's degree
Teaching English as a Second or Foreign Language/ESL Language Instructor:College/CEGEP
Teaching English as a Second or Foreign Language/ESL Language Instructor:Bachelor's degree
Teaching English as a Second or Foreign Language/ESL Language Instructor:Master's degree
Teaching English or French as a Second or Foreign La

Vehicle Maintenance and Repair Technologies, Other:Other trades certificate or diploma
Vehicle Maintenance and Repair Technologies, Other:College/CEGEP
Veterinary Anatomy (Cert., MSc, PhD):Degree in medicine, dentistry, veterinary medicine or optometry
Veterinary Anatomy (Cert., MSc, PhD):Master's degree
Veterinary Anesthesiology Residency Programs:Degree in medicine, dentistry, veterinary medicine or optometry
Veterinary Anesthesiology Residency Programs:Master's degree
Veterinary Biomedical and Clinical Sciences (Cert., MSc, PhD), Other:Degree in medicine, dentistry, veterinary medicine or optometry
Veterinary Biomedical and Clinical Sciences (Cert., MSc, PhD), Other:Master's degree
Veterinary Dentistry Residency Programs:Degree in medicine, dentistry, veterinary medicine or optometry
Veterinary Dentistry Residency Programs:Master's degree
Veterinary Dermatology Residency Programs:Degree in medicine, dentistry, veterinary medicine or optometry
Veterinary Dermatology Residency Program

Work and Family Studies:Other trades certificate or diploma
Work and Family Studies:College/CEGEP
Work and Family Studies:Bachelor's degree
Work and Family Studies:Master's degree
Writing:College/CEGEP
Yoga Teacher Training/Yoga Therapy:Bachelor's degree
Yoga Teacher Training/Yoga Therapy:Master's degree
Youth Ministry:College/CEGEP
Youth Ministry:Bachelor's degree
Youth Ministry:Master's degree
Youth Services/Administration:Other trades certificate or diploma
Youth Services/Administration:College/CEGEP
Youth Services/Administration:Bachelor's degree
Youth Services/Administration:Master's degree
Zoological Medicine Residency Programs:Degree in medicine, dentistry, veterinary medicine or optometry
Zoological Medicine Residency Programs:Master's degree
Zoology/Animal Biology, Other:College/CEGEP
Zoology/Animal Biology, Other:Bachelor's degree
Zoology/Animal Biology, Other:Master's degree
Zoology/Animal Biology, General:College/CEGEP
Zoology/Animal Biology, General:Bachelor's degree
Zoolo

In [89]:
# Check and clean the data before saving

edu_df_clean = jobs_df.dropna(subset=['edu_level','degree','description','top_jobs'])

edu_categories = [s.split('found under ')[1] for s in edu_df_clean['description']]
edu_categories = [s.strip('.') for s in edu_categories]
edu_df_clean['education_groups'] = edu_categories

# All education groups have the same recommended jobs, so we only need to keep one
education_df = edu_df_clean[['edu_level','degree','education_groups','top_jobs','job_percent','top_job_links']].drop_duplicates(subset =['education_groups','edu_level']) 
job_name_df = education_df[['edu_level','education_groups','top_jobs']].explode('top_jobs')
percent_df = education_df[['job_percent']].explode('job_percent')

link_df = education_df[['top_job_links']].explode('top_job_links')
link_df['top_job_links'].fillna('', inplace=True)

link_section = [x.split(';')[0] for x in link_df['top_job_links']]

job_name_df['job_percent'] = pd.to_numeric(percent_df['job_percent'].str.strip('%'),errors='coerce')
job_name_df['top_job_links'] = link_section
job_name_df['link'] = [x.replace('/marketreport/summary-occupation','') for x in link_section]
job_name_df['identifier'] = job_name_df['education_groups'].str.cat(job_name_df['edu_level'],sep=": ")

job_name_df

,edu_level,education_groups,top_jobs,job_percent,top_job_links,link,identifier
0,College/CEGEP,Accounting,Accounting and related clerks,11.99,/marketreport/summary-occupation/14122/ca,/14122/ca,Accounting: College/CEGEP
0,College/CEGEP,Accounting,Financial auditors and accountants,11.23,/marketreport/summary-occupation/131/ca,/131/ca,Accounting: College/CEGEP
0,College/CEGEP,Accounting,Accounting technicians and bookkeepers,10.46,/marketreport/summary-occupation/24500/ca,/24500/ca,Accounting: College/CEGEP
0,College/CEGEP,Accounting,Administrative officers,3.92,/marketreport/summary-occupation/12462/ca,/12462/ca,Accounting: College/CEGEP
0,College/CEGEP,Accounting,Retail salespersons,3.25,/marketreport/summary-occupation/20599/ca,/20599/ca,Accounting: College/CEGEP
...,...,...,...,...,...,...,...
3014,Master's degree,Sociology,Administrative officers,2.63,/marketreport/summary-occupation/12462/ca,/12462/ca,Sociology: Master's degree
3014,Master's degree,Sociology,Social and community service workers,2.63,/marketreport/summary-occupation/5112/ca,/5112/ca,Sociology: Master's degree
3014,Master's degree,Sociology,Business development officers and marketing re...,2.63,/marketreport/summary-occupation/3916/ca,/3916/ca,Sociology: Master's degree
3014,Master's degree,Sociology,University professors and lecturers,1.97,/marketreport/summary-occupation/4707/ca,/4707/ca,Sociology: Master's degree


In [24]:
# Save to file 

edu_df_clean.to_csv(os.path.join(data_path,'processed','jobbank-fields.csv'), index=False)
job_name_df.to_csv(os.path.join(data_path,'processed','education-to-job.csv'), index=False)

In [25]:
# Load csv if not in memory. Note that saving as csv flattens lists to strings.

job_name_df = pd.read_csv(os.path.join(data_path,'processed','education-to-job.csv'))
edu_df_load = pd.read_csv(os.path.join(data_path,'processed','jobbank-fields.csv'))

# Use eval to turn lists of strings back into lists
edu_df = convert_to_list(edu_df_load,['unemployed',
                                      'unrelated_field',
                                      'relation',
                                      'top_jobs',
                                      'top_job_links',
                                      'job_percent'])

In [12]:
# Load full job_url_list from previous notebook
# Note that previously I was using the output from the education scraping,
# but this is a much smaller list of jobs

file_path = (os.path.join(data_path,'processed','job_url_list.txt'))

# define empty list
url_end = []

# open file and read the content in a list
with open(file_path, 'r') as filehandle:
    url_end = [current_place.rstrip() for current_place in filehandle.readlines()]

# Scrape individual job data

In [72]:
# Go to each valid job link and scrape data there.

base_url = 'https://www.jobbank.gc.ca/marketreport/'

noc_list = []

overview_df = pd.DataFrame({'link':[],
                            'noc':[],
                            'job_title':[],
                            'salary':[]})

description_df = pd.DataFrame({'noc':[],
                               'description':[],
                               'duties':[],
                               'titles':[],
                               'related_occupations':[]})

regulation_df = pd.DataFrame({'noc':[],
                              'province':[],
                              'regulation':[]})

skills_df = pd.DataFrame({'noc':[],
                          'expertise':[],
                          'skills':[],
                          'knowledge':[]})


for link in url_end:
    
    try:
        # Main summary page
        url = base_url + 'summary-occupation' + link
        with requests.get(url, timeout=5, headers=headers) as u:     
            soup = BeautifulSoup(u.content) 

            # the following is very specific to the website
            a = soup.find('div', attrs={'class':'job-market'})
            
            # Add any interested attributes here
            title = a.find('span', attrs={'class':'heading-info'})
            intro = a.find('p', attrs={'class':'intro'})
            
            wages_section = a.find('a', attrs={'class':'wages-section'})
            salary = wages_section.find('p', attrs={'class':'section-value'})

            if title is not None and intro is not None:
                job_title = title.text.strip().replace(' in Canada','')
                job_intro = intro.text.strip()
                noc_text = job_intro.split('NOC ')[1]
                noc_code = noc_text.strip(').')
                
                print(noc_code + ': ' + job_title)
            else:
                continue

            if salary:
                job_salary = salary.text.strip()
            else:
                job_salary = 'NaN'

            overview_df = overview_df.append({'link': link,
                                              'noc': noc_code,
                                              'job_title': job_title,
                                              'salary': job_salary}, ignore_index=True)
            
    except:
        continue
        
    # For a given noc code, all the skills and other data is the same,
    # therefore we only need to collect for new noc codes
    
    if noc_code not in noc_list:
        
    
        try:    
            # Description page
            url = base_url + 'occupation' + link
            with requests.get(url, timeout=5, headers=headers) as u:     
                soup = BeautifulSoup(u.content) 

                desc_head = soup.find('h3', text = 'Description')
                if desc_head.findNext('p'):
                    description = desc_head.findNext('p')
                    job_description = description.text.strip()
                else:
                    description = 'NaN'

                job_duties = []
                duty_head = soup.find('h3', text = 'Job duties')
                duty_list = duty_head.findNext('li')
                for sib in duty_list.next_siblings:
                    if sib.name == 'li':
                        job_duties.append(sib.text.strip())
                    else:
                        break

                alt_titles = []
                title_head = soup.find('h3', text = 'Job titles').findNext('ul')
                title_bullets = title_head.findAll('li')
                if title_bullets:
                    alt_titles = [s.text.strip() for s in title_bullets]
                else:
                    alt_titles = 'NaN'

                rel_jobs = []
                related_head = soup.find('h3', text = 'Related occupations').findNext('ul')
                related_list = related_head.findAll('li')
                if related_list:
                    rel_jobs = [s.text.strip() for s in related_list]
                else:
                    rel_jobs = 'NaN'

                description_df = description_df.append({'noc': noc_code,
                                                        'description':job_description,
                                                        'duties':job_duties,
                                                        'titles':alt_titles,
                                                        'related_occupations':rel_jobs}, ignore_index=True)
                
                # Add noc code to the list so we don't scrape next time
                # only if we get the far
                noc_list.append(noc_code)
                
        except:
            continue

    # Outlook page
    #         url = base_url + 'outlook-occupation' + link
    #         with requests.get(url, timeout=5, headers=headers) as u:     
    #             soup = BeautifulSoup(u.content) 

    #             # the following is very specific to the website
    #             a = soup.find('div', attrs={'class':'job-market'})

    #             #try:
    #             # Add any interested attributes here
    #             prov_sec = a.findAll('td', attrs={'headers':'header1_outlook_prov'})
    #             for prov in prov_sec:
    #                 on_href = a.find('a', text = 'Ontario')
    #                 if on_href:
    #                     prospect = on_href.findNext('span', attr = {'class':'outlooknote'})
    #                     job_prospects = prospect.text.strip()
    #                 else:
    #                     job_prospects = 'NaN'

        try:
            
            # Certification page
            url = base_url + 'requirements' + link
            with requests.get(url, timeout=5, headers=headers) as u:     
                soup = BeautifulSoup(u.content) 
                province = []
                regulation = []
                for tr in soup.find_all('tr')[1:]:
                    tds = tr.find_all(recursive=False)
                    if tds:
                        province.append(tds[0].text.strip('\n \t'))
                        regulation.append(tds[2].text.strip('\n \t'))
                    else:
                        province = 'NaN'
                        regulation = 'NaN'

                regulation_df = regulation_df.append({'noc': noc_code,
                                                      'province':province,
                                                      'regulation':regulation}, ignore_index=True)                    
        except:
            continue

        try:

            # Skills page
            url = base_url + 'skills' + link
            with requests.get(url, timeout=5, headers=headers) as u:     
                soup = BeautifulSoup(u.content)

                #Expertise
                expert = []
                expert_head = soup.find('h2', text = 'Expertise').findNext('ul')
                expert_bullets = expert_head.findAll('li')
                if expert_bullets:
                    expert_list = [s.text.strip() for s in expert_bullets]
                else:
                    expert_list = 'NaN'

                #Skills
                skills = []
                skills_sec = soup.find('ul', attrs={'class':'sk'})
                skills_list = skills_sec.find('li')

                if skills_list:
                    skills_groups = [s.text.replace('\t','') for s in skills_list.find_next_siblings('li')]
                    for group in skills_groups:
                        skill_txt = group.split('\n')
                        skill_simple = [s.strip() for s in skill_txt if not 'Help - ' in s]
                        skill_simple = [s for s in skill_simple if s]
                        for skill in skill_simple[1:]:
                            skills.append(skill_simple[0].strip() + ' : ' + skill.strip())
                else:
                    skills = 'NaN'

                #Knowledge
                knowledge = []
                know_sec = skills_sec.findNext('ul', attrs={'class':'sk'})
                know_list = know_sec.find('li')

                if know_list:
                    know_groups = [s.text.replace('\t','') for s in know_list.find_next_siblings('li')]
                    for group in know_groups:
                        know_txt = group.split('\n')
                        know_simple = [s.strip() for s in know_txt if not 'Help - ' in s]
                        know_simple = [s for s in know_simple if s]
                        for know in know_simple[1:]:
                            knowledge.append(know_simple[0].strip() + ' : ' + know.strip())
                else:
                    know = 'NaN'

                skills_df = skills_df.append({'noc': noc_code,
                                              'expertise':expert_list,
                                              'skills':skills,
                                              'knowledge':knowledge}, ignore_index=True)

        except:
            continue



7205: Construction Painters Foreman/woman
7205: Asphalt Roofing Contractor
7205: Bricklaying And Masonry Contractor
7205: Contractor, Decorating
7205: Contractor, Drywall Application
7205: Contractor, Floor Covering
7205: Contractor, Glazing
7205: Contractor, Insulating
7205: Painting Contractor
7205: Contractor, Paperhanging
7205: Contractor, Plastering
7205: Roofing Contractor
7205: Contractor, Stonemasonry
7205: Contractor, Terrazzo Finishing
7205: Contractor, Tile, Terrazzo And Marble Setting
7205: Contractor, Tilesetting
7205: Contractor, Wood Shingle Roofing
7205: Customer Service Supervisor - Housing Construction
7205: Decorators Foreman/woman
7205: Eavestrough Installation Contractor
7205: Exterminating Service Supervisor
7205: Contractor, Fencing
7205: Floor Covering Foreman/woman
7205: Floor Covering Supervisor
7205: Asphalt Roofers Foreman/woman
7205: Boiler And Pipe Insulators Foreman/woman
7205: Building Insulators Foreman/woman
7205: Concrete Finishers Foreman/woman
7205:

7243: Relay Tester And Repairer - Electrical Power System
7243: Substation Electrician - Electrical Power System
7243: Power System Electrician
7243: Electrician - Electricity Production
7243: Electrical Powerhouse Electrician - Electrical Power System
7243: Electrician, Electrical Substation - Electrical Power System
7243: Power Electrician
7244: Power Lineman/woman
7244: Apprentice Line Maintainer - Electric Power Systems
7244: Apprentice Lineman/woman - Electric Power Systems
7244: Assistant Transmission Lineman/woman - Electric Power Systems
7244: Cable Installer And Repairer - Electric Power Systems
7244: Meter Installer - Electric Power Systems
7244: Cable Repairer - Electric Power Systems
7244: Cable Splicer - Electrical Power
7244: Line Servicer - Electric Power Systems
7244: Cable Installer - Electrical Power
7244: Conduit Installer - Electric Power Systems
7244: Construction Lineman/woman - Electric Power Systems
7244: Construction Lineman/woman - Electrical Power Line
7244: 

9227: Spikemaking Foreman/woman
9227: Sporting Equipment Manufacturing Foreman/woman
9227: Thermostat Assembly Foreman/woman
9227: Foreman/woman, Toy Assembly
9227: Foreman/woman, Wooden Truss Makers - Manufacturing
9227: Foreman/woman, Wooden Box Maker
9227: Woodenware Assembler Foreman/woman
9227: Foreman/woman, Hand Tool Manufacturing
9227: Foreman/woman, Optical Instrument Assembly
9227: Bicycle Assembly Supervisor
9227: Clock And Watch Assembly Supervisor
9227: Jewellery Assembly Supervisor
9227: Lens Manufacturing (non-prescription) Supervisor
9227: Supervisor, Millwork Assembly
9227: Supervisor, Silverware Manufacturing
9227: Sports Equipment Assembly Supervisor
9227: Sporting Goods Manufacturing Supervisor
9227: Supervisor, Toy Manufacturing
9227: Clock And Watch Assembly Repairers Supervisor
9227: Foreman/woman, Toy Manufacturing
9227: Clock And Watch Assembly Repairers Foreman/woman
9227: Wood Products Foreman/woman
9235: Control Operator, Paper Machine
9235: Cylinder Machine

7315: Aircraft Maintenance Engineer (AME) (except Avionics)
7315: Aircraft Engine Repair And Overhaul Mechanic
7315: Aircraft Gas Turbine Engine Technician
7315: Aircraft Reciprocating Engine Technician
7315: Aircraft Structural Repair Technician
7315: Aviation Mechanical Component Shop Technician
7315: Composite Repair Technician - Aircraft
7315: Flight Line Mechanic
7315: Gas Turbine Engine Mechanic
7315: Structural Repair Technician
7315: Structures Mechanic
7315: Aircraft Propeller Systems Technician
7315: Aircraft Mechanic Learner
7315: Aircraft Maintenance Mechanic Apprentice
7315: Aviation Maintenance Inspector
7316: Pulp And Paper Machinery Builder
7316: Chemical-processing Machine Builder
7316: Heavy Equipment Fitter
7316: Locomotive Builder
7316: Machine Fitter
7316: Machine Builder
7316: Aircraft Engine Fitter
7316: Assembly Fitter
7316: Aircraft Engine Assembly Fitter
7316: Mechanical Fitter
7316: Machine Builder-fitter
7316: Textile Machinery Fitter
7316: Turbine Fitter
73

9241: Thermal Plant Operator
9241: Energy Recovery Incinerator Plant Operator
9241: Plant Maintenance Stationary Engineer
9241: Refrigeration Plant Operator
9241: Heating Stationary Engineer
9241: Stationary Operating Engineer
9241: Stationary Engineer "A" And Refrigeration Plant Operator
9241: Stationary Engineer "B" And Refrigeration Plant Operator
9241: Stationary Power Engineer
9241: Power Engineer
7361: Switch Engineer - Railway
7361: Locomotive Engineer - Railway
7361: Diesel Locomotive Engineer - Railway
7361: Yard Engineer - Railway
7361: Freight Train Engineer
7361: Passenger Train Locomotive Engineer
7361: Transfer Hostler - Railway
7361: Engineer Trainee, Railway
7361: Industrial Locomotive Engineer
7361: Train Engineer
7361: Railway Engineer
7361: Railway Locomotive Engineer
7362: Brake Worker - Switching Yard
7362: Railway Conductor
7362: Freight Train Conductor
7362: Passenger Train Conductor
7362: Sleeping Car Conductor
7362: Pullman Conductor
7362: Conductor - Railway Y

...

6232: Real Estate Broker
6232: Real Estate Sales Consultant
6232: Residential Real Estate Agent
6232: Real Estate Agency Representative
6232: Real Estate Agent Supervisor
6232: Real Estate Sales Supervisor
6222: Appliance Buyer
6222: Assistant Buyer
6222: Buyer - Retail
6222: Cattle Buyer
6222: Clothing Buyer
6222: Food Buyer
6222: Fruit Buyer
6222: Fur Buyer
6222: Hardware Buyer
6222: Hog Buyer
6222: Livestock Buyer
6222: Log Buyer
6222: Lumber Buyer
6222: Meat Buyer
6222: Merchandiser
6222: Pharmaceuticals Buyer
6222: Produce Buyer
6222: Retail Merchandiser
6222: Salvage Buyer
6222: Scrap Buyer
6222: Chief Buyer
6222: Buyers Supervisor
6222: Beverage Taster And Buyer
6222: Timber Buyer
6222: Tobacco Buyer
6222: Wholesale Buyer
6222: Perishable Food Buyer
6222: Specialty Food Buyer
6222: Buyer - Wholesale
6322: Apprentice Cook
6322: Journeyman/woman Cook
6322: Licensed Cook
6322: Cook
6322: Small Establishment Cook
6322: Institutional Cook
6322: Hospital Cook
6322: Banquet Cook
6322: 

7282: Cement Mason, Maintenance
7282: Concrete Finisher
7282: Concrete Floor Finisher
7282: Concrete Mason
7282: Concrete Patcher-finisher
7282: Precast Concrete Finisher
7282: Concrete Smoother-finisher
7282: Journeyman/woman Cement Finisher
7282: Sidewalk Concrete Finisher
7282: Cement Mason
7282: Finisher, Concrete Products
7282: Concrete Polisher
7282: Cement Mason Apprentice
7282: Concrete Finisher Apprentice
7292: Glazier Apprentice
7292: Journeyman/woman Glazier
7292: Plate Glass Installer
7292: Mirror Wall Installer
7292: Curtain Wall Glazier
7292: Glass Setter - Construction
7292: Glazier, Window Sash And Frame
7292: Stained Glass Glazier
7292: Glass And Metal Mechanic
7292: Structural Glass Glazier
7292: Exterior Wall Glazier
7292: Building Glazier
7292: Leaded Glass Glazier
7292: Glass Installer-glazier
7292: Glass Wall Installer
7292: Glazier
7292: Installer, Structural Glass
7292: Glass Wallcovering Installer
7292: Glazier And Metal Mechanic
7292: Glass Worker - Constructi

7301: Foreman/woman, Aircraft Mechanics And Inspectors
7301: Aircraft Mechanics And Inspectors Foreman/woman - Mechanical Systems
7301: Assembly Mechanics Foreman/woman
7301: Automobile Mechanics Foreman/woman
7301: Automotive Service Mechanics Foreman/woman
7301: Foreman/woman, Electrical Mechanics
7301: Elevator Mechanic Foreman/woman
7301: Foreman/woman, Furnace Installers
7301: Foreman/woman, Heating Systems Mechanics
7301: Foreman/woman, Heavy Agricultural Machinery Assembly
7301: Foreman/woman, Heavy Construction Equipment Assembly
7301: Foreman/woman, Heavy Equipment Assembly
7301: Foreman/woman, Heavy Farm Equipment Assembly
7301: Foreman/woman, Heavy Machinery Assembly
7301: Foreman/woman, Helicopter Maintenance - Mechanical Systems
7301: Foreman/woman, Industrial Machinery Manufacturing
7301: Foreman/woman, Industrial Mechanics
7301: Foreman/woman, Industrial, Farm And Construction Machinery Mechanics And Repairers
7301: Foreman/woman, Locomotive Inspectors
7301: Foreman/woma

7205: Building Maintenance Supervisor
7205: Cement Finisher Foreman/woman
7205: Cement Finishing Contractor
7205: Asbestos Removal Supervisor
7441: Blinds Assembler-installer
1112: Change Management Consultant
0013: Chief Technology Officer (CTO)
2171: Cloud Architect — Information Technology (IT)
4151: Early Intervention Therapist
1226: Event Decorator
1111: Junior Accountant
6732: Kitchen Exhaust Cleaner
6221: Outside Sales Representative
7611: Renovation Helper
4214: Early Childhood Substitute Teacher
0651: Tattoo Studio Manager 
0711: Transportation Construction Manager 
0511: University Librarian
5131: Video Game Writer
6733: Construction Cleaner
7611: Disaster Restoration Technician
3114: Emergency Veterinarian 
7205: Exterior Insulation And Finish Systems (EIFS) Supervisor
4212: First Nations Criminal Court Worker
7231: Manual Machinist
1112: Mergers And Acquisitions Associate
4021: Supply Instructor
2251: Truss Designer-estimator
4212: Victim Services Counsellor
4153: Women's C

0013: Compensation Vice-president – Financial, Communications And Other Business Services
0014: Compensation Vice-president – Health, Education, Social And Community Services And Membership Organizations
0015: Compensation Vice-president – Trade, Broadcasting And Other Services
0016: Compensation Vice-president – Goods Production, Utilities, Transport And Construction
0821: Cash Crop Farmer
0821: Sow Farm Manager
0823: Aquaculture Director
0823: Aquaculture Program Director
0821: Hatchery Supervisor
0823: Shellfish Farmer
0016: Project Executive – Construction
0423: Child-care Centre Director
0423: Preschool Director
0423: Preschool Program Director
0421: Academic Affairs Vice-president – Post-secondary Education
0422: Academic Dean – High School
0421: Academic Dean – College Or University
0421: Financial Aid Director – Post-secondary Education
0421: Institutional Research Director
0013: Chief Learning Officer (CLO) – Financial, Communications And Other Business Services
0014: Chief Le

In [26]:
# Skills are confined to a list of options

skills_expand = skills_df.explode('skills')
print(len(skills_expand))
#Number of unique skills
len(sorted(skills_expand['skills'].unique()))

3320


44

# Clean dataframes before saving

In [27]:
# Clean overview_df
overview_df['salary'] = [s.strip('$').replace('/hour','') for s in overview_df['salary']]
overview_df['salary'] = pd.to_numeric(overview_df['salary'],errors='coerce')

# Clean regulation_df
reg_df = regulation_df[['noc','province']].explode('province')
reg_explode = regulation_df[['regulation']].explode('regulation')

reg_df['regulation'] = reg_explode


In [88]:
overview_df.to_csv(os.path.join(data_path,'processed','job-overview.csv'), index=False)
description_df.to_csv(os.path.join(data_path,'processed','job-description.csv'), index=False)
reg_df.to_csv(os.path.join(data_path,'processed','job-regulation.csv'), index=False)
skills_df.to_csv(os.path.join(data_path,'processed','job-skills.csv'), index=False)

# Add noc_code to job_name_df for link between all DFs

In [31]:
# Load DFs if not in memory

overview_df = pd.read_csv(os.path.join(data_path,'processed','job-overview.csv'))
job_name_df = pd.read_csv(os.path.join(data_path,'processed','education-to-job.csv'))

In [42]:
job_noc_df = pd.merge(left=job_name_df, 
                      right=overview_df[['link','noc']].astype(str), 
                      how='left', 
                      left_on='link', 
                      right_on='link')

In [43]:
# Save updated DF to file

job_noc_df.to_csv(os.path.join(data_path,'processed','education-to-job.csv'), index=False)
